In [ ]:
! pip install python-dotenv
! pip install pydub
! pip install pandas
! pip install psycopg2
! pip install google-auth google-auth-httplib2 google-api-python-client google-auth-oauthlib
! pip install tqdm
! pip install pydub

In [ ]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import download_audio_gdrive


In [ ]:
from pydub import AudioSegment
import pandas as pd

# Function to get audio duration from an audio file
def get_audio_duration(audio_filename):
    try:
        audio = AudioSegment.from_file(audio_filename)
        duration_in_sec = len(audio) // 1000  # Duration in seconds
        duration = pd.to_datetime(duration_in_sec, unit='s').strftime('%H:%M:%S')  # Duration in H:M:S format
        return duration
    except Exception as e:
        print(f"Error fetching duration for {audio_filename}: {e}")
        return None


In [ ]:
from_id = 305
to_id = 319

durations = []

df = read_spreadsheet(sheet_id="107pF1LcHgbJtCGCdGwffqIyM70eImBmgPOjUuHQfZ7g")
for index, row in df.iterrows():
    if not isinstance(row['Audio'], str) or not isinstance(row['STT_0000'], str):
        break
    id = row['STT_0000']
    gd_url = row['Audio']
    sr_no = row.iloc[0]
    if sr_no >= from_id and sr_no <= to_id:
        print(id, gd_url)
        id = id + '.wav'
        download_audio_gdrive(gd_url, id)
        audio_filename = f"full_audio/{id}"
        duration = get_audio_duration(audio_filename)
        print(f"Audio duration for {audio_filename}: {duration}")
        durations.append((id, duration))


In [ ]:
# Convert list to DataFrame and save to CSV
df_durations = pd.DataFrame(durations, columns=['File Name', 'Duration'])
df_durations.to_csv('nw_gh_audio_durations.csv', index=False)

print("Durations have been saved to audio_durations.csv")